<a href="https://colab.research.google.com/github/HankCaswell/SolarCompetitiveIntel/blob/main/NC_Solar_Plants.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir("/content/drive/My Drive/STAR/Data")
!ls -l

Mounted at /content/drive
total 1533
-rw------- 1 root root 850537 Jan 29 20:48 plants.json
-rw------- 1 root root 718664 Jan 21 23:54 uspvdb_v2_0_20240801.csv


In [3]:
import pandas as pd

# Load the JSON file into a pandas DataFrame.
df = pd.read_json('plants.json')

# Now you can work with the DataFrame 'df'. For example, to display the first few rows:
print(df.head())

   Plant Code                        Plant Name  Utility Code  \
0       62873  Carolina Poultry Power Farmville         62749   
1        2727                     Marshall (NC)          5416   
2        2721    James E. Rogers Energy Complex          5416   
3        2718                         G G Allen          5416   
4        6250                              Mayo          3046   

                  Utility Name         Plant Address Plant City  \
0       Carolina Poultry Power  8966 W Marlboro Road  Farmville   
1   Duke Energy Carolinas, LLC     8320 East Hwy 150    Terrell   
2   Duke Energy Carolinas, LLC   Duke Power Rd #1002  Cliffside   
3   Duke Energy Carolinas, LLC    253 Plant Allen Rd    Belmont   
4  Duke Energy Progress - (NC)     10660 Boston Road    Roxboro   

       Plant County Plant State  Plant Zip  Plant Latitude  Plant Longitude  \
0       Pitt County          NC      27828       35.591719       -77.614507   
1    Catawba County          NC      28682      

In [4]:
# prompt: I want to take the json file that I just accessed, and filter it only by "Fuel Types" and return a CSV of only the entries that use "Solar"

# Filter the DataFrame to include only rows where 'Fuel Types' contains 'Solar'.
solar_plants = df[df['Fuel Types'].str.contains('Solar', na=False)]

# Save the filtered data to a CSV file.
solar_plants.to_csv('solar_plants.csv', index=False)

# Download the CSV file.
from google.colab import files
files.download('solar_plants.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
import pandas as pd
import plotly.express as px

# Assuming 'solar_plants.csv' is in the current working directory
# If not, adjust the path accordingly.
df = pd.read_csv('solar_plants.csv')

# Create the scatter_mapbox plot
# The change is in the hover_data argument below
fig = px.scatter_mapbox(df,
                        lat="Plant Latitude",
                        lon="Plant Longitude",
                        hover_name="Plant Name",
                        hover_data=["Annual Generation", "Fuel Types", "Plant State"], # Changed "State" to "Plant State"
                        color="Annual Generation",  # Color points by Annual Generation
                        size="Annual Generation",  # Size points by Annual Generation
                        zoom=6,  # Adjust zoom level as needed
                        mapbox_style="carto-positron", # or open-street-map
                        center={"lat": 35.5, "lon": -79}, # Center map on North Carolina
                        width=800,  # Adjust width
                        height=600) # Adjust height


fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [15]:
# prompt: That map looks good, I want to now be able to filter the points that are showing up on the map based on the column "Utility Name" which is in 'solarplants.csv'.  Additionally, I want to add a column to the data in 'solarplants.csv',  "Utility Type", which I would like to integrate from data stored in the folder we've been working out of called "powercompanies.json". Ideally, after a user has filtered by "Utility Type" and "Utility Name", I would like for them to be able to download a csv of the data that is displaying on the map.

from google.colab import drive
import os
import pandas as pd
from google.colab import files
import plotly.express as px
import json

drive.mount('/content/drive')

os.chdir("/content/drive/My Drive/STAR/Data")
!ls -l

# Load the JSON file into a pandas DataFrame.
df = pd.read_json('plants.json')

# Filter for solar plants
solar_plants = df[df['Fuel Types'].str.contains('Solar', na=False)]
solar_plants.to_csv('solar_plants.csv', index=False)

# Load solar plant data and power company data
solar_df = pd.read_csv('solar_plants.csv')
with open('powercompanies.json', 'r') as f:
    power_companies = json.load(f)

# Convert the JSON data to a DataFrame
power_companies_df = pd.DataFrame(power_companies)

# Merge the two dataframes based on 'Utility Name' and 'Company Name'
solar_df = pd.merge(solar_df, power_companies_df[['Company Name', 'Utility Type']], left_on='Utility Name', right_on='Company Name', how='left')

# Drop the redundant 'Company Name' column
solar_df = solar_df.drop(columns=['Company Name'])

# Fill any NaN values in 'Utility Type' with 'Unknown'
solar_df['Utility Type'].fillna('Unknown', inplace=True)

# Create the scatter_mapbox plot
fig = px.scatter_mapbox(solar_df,
                        lat="Plant Latitude",
                        lon="Plant Longitude",
                        hover_name="Plant Name",
                        hover_data=["Annual Generation", "Fuel Types", "Plant State", "Utility Name", "Utility Type"],
                        color="Annual Generation",
                        size="Annual Generation",
                        zoom=6,
                        mapbox_style="carto-positron",
                        center={"lat": 35.5, "lon": -79},
                        width=800,
                        height=600)

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

# Add interactive filtering (requires ipywidgets)
import ipywidgets as widgets
from IPython.display import display

# Utility Name filter
utility_name_dropdown = widgets.Dropdown(
    options=solar_df['Utility Name'].unique().tolist(),
    value=solar_df['Utility Name'].unique().tolist()[0],
    description='Utility Name:'
)


# Utility Type filter
utility_type_dropdown = widgets.Dropdown(
    options=solar_df['Utility Type'].unique().tolist(),
    value=solar_df['Utility Type'].unique().tolist()[0],
    description='Utility Type:'
)


def update_map(change):
    selected_utility_name = utility_name_dropdown.value
    selected_utility_type = utility_type_dropdown.value

    filtered_df = solar_df[
        (solar_df['Utility Name'] == selected_utility_name) & (solar_df['Utility Type'] == selected_utility_type)
    ]

    new_fig = px.scatter_mapbox(filtered_df,
                        lat="Plant Latitude",
                        lon="Plant Longitude",
                        hover_name="Plant Name",
                        hover_data=["Annual Generation", "Fuel Types", "Plant State", "Utility Name", "Utility Type"],
                        color="Annual Generation",
                        size="Annual Generation",
                        zoom=6,
                        mapbox_style="carto-positron",
                        center={"lat": 35.5, "lon": -79},
                        width=800,
                        height=600)

    new_fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    new_fig.show()

    #Download button
    @widgets.interact(filename="filtered_data.csv")
    def download_csv(filename):
        filtered_df.to_csv(filename, index=False)
        files.download(filename)



utility_name_dropdown.observe(update_map, names='value')
utility_type_dropdown.observe(update_map, names='value')

display(utility_name_dropdown)
display(utility_type_dropdown)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
total 9015
-rw------- 1 root root  268327 Jan 29 21:23 filtered_data.csv
-rw------- 1 root root  850537 Jan 29 20:48 plants.json
-rw------- 1 root root 7119234 Jan 29 21:04 powercompanies.json
-rw------- 1 root root  273074 Jan 29 21:20 solar_plants.csv
-rw------- 1 root root  718664 Jan 21 23:54 uspvdb_v2_0_20240801.csv


<ipython-input-15-0cc15fdda5ca>:37: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





Dropdown(description='Utility Name:', options=('White Street Renewables LLC', 'Fayetteville Public Works Commi…

Dropdown(description='Utility Type:', options=('Independent Power Producer', 'Municipally-Owned Utility', 'Unk…

In [14]:
# prompt: Almost there, I need an option to filter only by utility type, that is, I want an option to show all under "utility name" while only looking at certain subsections of utility type

# Previous code remains the same up to the interactive filtering section

# Add interactive filtering (requires ipywidgets)

# Utility Type filter
utility_type_dropdown = widgets.Dropdown(
    options=solar_df['Utility Type'].unique().tolist(),
    value=solar_df['Utility Type'].unique().tolist()[0],
    description='Utility Type:'
)

def update_map(change):
    selected_utility_type = utility_type_dropdown.value
    filtered_df = solar_df[solar_df['Utility Type'] == selected_utility_type]

    #Update Utility Name dropdown based on the selected Utility Type
    utility_name_dropdown.options = filtered_df['Utility Name'].unique().tolist()
    if utility_name_dropdown.options:
      utility_name_dropdown.value = utility_name_dropdown.options[0]
    else:
      # Handle case where no utility names are available after filtering
      print("No utility names found for the selected utility type.")
      return

    new_fig = px.scatter_mapbox(filtered_df,
                        lat="Plant Latitude",
                        lon="Plant Longitude",
                        hover_name="Plant Name",
                        hover_data=["Annual Generation", "Fuel Types", "Plant State", "Utility Name", "Utility Type"],
                        color="Annual Generation",
                        size="Annual Generation",
                        zoom=6,
                        mapbox_style="carto-positron",
                        center={"lat": 35.5, "lon": -79},
                        width=800,
                        height=600)

    new_fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    new_fig.show()

    #Download button
    @widgets.interact(filename="filtered_data.csv")
    def download_csv(filename):
        filtered_df.to_csv(filename, index=False)
        files.download(filename)


# Utility Name filter (now dependent on Utility Type)
utility_name_dropdown = widgets.Dropdown(
    options=solar_df['Utility Name'].unique().tolist(),
    value=solar_df['Utility Name'].unique().tolist()[0],
    description='Utility Name:'
)

utility_type_dropdown.observe(update_map, names='value')
utility_name_dropdown.observe(update_map, names='value')

display(utility_type_dropdown)
display(utility_name_dropdown)

Dropdown(description='Utility Type:', options=('Independent Power Producer', 'Municipally-Owned Utility', 'Unk…

Dropdown(description='Utility Name:', options=('White Street Renewables LLC', 'Fayetteville Public Works Commi…

interactive(children=(Text(value='filtered_data.csv', description='filename'), Output()), _dom_classes=('widge…

interactive(children=(Text(value='filtered_data.csv', description='filename'), Output()), _dom_classes=('widge…